In [107]:
import pandas as pd
import numpy as np
import re

In [108]:
dbfile = '../sfcompoDB.pkl'
fulldb = pd.read_pickle(dbfile)

### Steal some functions/definitions from K. Dayman's sfcompo.py: can refactor later if needed

In [91]:
# Conversion Factors
cF = {'GW*d/tHMi': {'MW*d/tUi': 1000.0},
      'GW*d/tUi': {'MW*d/tUi': 1000.0},
      'MW*d/kgUi': {'MW*d/tUi': 1000.0},
      'MW*d/tHMi': {'MW*d/tUi': 1.0},
      'MW*d/tUi': {'MW*d/kgUi': 0.001, 'MW*d/tUi': 1.0},
      'MW*h/kgUi': {'MW*d/tUi': 41.66666667},
      'g/gFueli', 
      'mg/gFueli', 
      'kg/tUi', 
      'mol/gFueli', 
      'mg/gUi',
      'mol/molUf', 
      'mol/molU238f', 
      'kg/kgUf', 
      'kg/kgPuf', 
      'mol/molPuf',
      'g/gUi', 
      'g/gU238f', 
      'g/tUi', 
      'mg/gUf', 
      'g/tHMi', 
      'mol/molU238i',
      'g/gHMi', 
      'mol/molUi', 
      'μg/gUi', 
      'μg/gUf', 
      'mg/gU238f', 
      'g/gU238i',
      'g/gUf', 
      'mol/molHMi', 
      'mg/kgUi', 
      'mg/tUi', 
      'g/kgUi', 
      'g/gPuf',
      'mg/gHMi',
      #'Ci/gUi'
      'Bq/gUi': {'Ci/gUi': 2.7e-11}, 
      'Bq/kgUi': {'Ci/gUi': 2.7e-14},
      'Bq/gUf': {'Ci/gUf': 2.7e-11}, 
      'Ci/gFueli': {'Ci/gUi': 1.0}, 
      'Ci/gHMi': {'Ci/gUi': 1.0},
      'Pa': {'bar': 1e-05, 'psi': 0.000145038},
      'bar': {'Pa': 100000.0, 'psi': 14.5038},
      'cm': {'in': 0.393701, 'm': 0.01, 'mm': 10.0},
      'g': {'kg': 0.001, 'lb': 0.00220462, 'mg': 1000.0},
      'in': {'cm': 2.54, 'm': 0.0254, 'mm': 25.4},
      'kg': {'g': 1000.0, 'lb': 2.20462, 'mg': 1000000.0},
      'lb': {'g': 453.592, 'kg': 0.453592, 'mg': 453592.0},
      'm': {'cm': 100.0, 'in': 39.3701, 'mm': 1000.0},
      'mg': {'g': 0.001, 'kg': 1e-06, 'lb': 2.20462e-06},
      'mm': {'cm': 0.1, 'in': 0.0393701, 'm': 0.001},
      'psi': {'Pa': 689476.0, 'bar': 0.0689476}
      }

def changeUnits_(row, rv, ru, u, cF):
    '''
    Internal function used by changeUnits under Pandas.apply()
    '''
    cur_unit = u[int(row['idx'])]
    cf = getCF(cur_unit, ru, cF)
    row[rv] = row[rv] * cf
    return row


def changeUnits(dF, requestedValue, requestedUnit, U, cF):
    '''
    Given an SFCompo dF, the requestedValue (column) is extracted, and all
    values in that column are converted to the requestedUnit before returning.
    ConversionFactor table is queried and user is prompted as necessary.
    '''
    idx_col = dF.columns.get_loc(requestedValue)
    u = U[:, idx_col]
    dF['idx'] = np.arange(dF.shape[0])
    X = dF[['idx', requestedValue]].copy()
    X = X.apply(lambda x: changeUnits_(x, requestedValue, requestedUnit, u, cF), axis=1)
    X = X.drop('idx', axis=1)
    return X

In [109]:
def getUnit(s):
    '''
    Given a string, finds the numeric value and remainder, presumably a unit
    indication. Returns a float and a string.
    '''
    numeric = '0123456789-.'
    for i,c in enumerate(s):
        if c not in numeric:
            break
    number = s[:i]
    unit = s[i:].strip()
    return number, unit

def extractUnits(df, col):
    '''
    Iterates over column in a dataframe, extracting values and units.
    Returns a dataframe with two separated columns of values and units.
    '''
    units = []
    def f(x):
        value, unit = getUnit(x)
        units.append(unit)
        return value
    
    df[col] = df[col].apply(lambda x: f(x))
    df[col+'Units'] = units
    return df

In [110]:
fulldb.drop(columns=['Sample identifier', 'Z', 'A', 'I'], inplace=True)

In [111]:
cols = {'Reactor name' : 'ReactorName', 
        'Reactor type' : 'ReactorType',
        'Estimated burnup' : 'Burnup', 
        'e. U235' : 'Enrichment',
        'SFCompo sample ref' : 'SampleRef',
        'Measurement type' : 'Measurement', 
        'Item' : 'Isotope',
        'Value' : 'Value', 
        'Unit' : 'Unit',
        'Concentration' : 'Concentration', 
        'Concentration Unit' : 'ConcentrationUnit',
        'Sigma' : 'Sigma', 
        'Uncertainty' : 'Uncertainty'
        }

In [112]:
fulldb.rename(columns=cols, inplace=True)
fulldb = extractUnits(fulldb, 'Burnup')
fulldb = extractUnits(fulldb, 'Enrichment')

In [113]:
fulldb.Measurement.unique()

array(['Activity', 'Isotopic Concentration', 'Mass Ratio', 'Atom Ratio',
       'Burnup', 'Other', 'Activity Ratio', 'Depletion'], dtype=object)

In [114]:
conc = fulldb.loc[fulldb['Measurement'] == 'Isotopic Concentration'].copy()
conc.Unit.unique()

array(['g/gFueli', 'mg/gFueli', 'kg/tUi', 'mol/gFueli', 'mg/gUi',
       'mol/molUf', 'mol/molU238f', 'kg/kgUf', 'kg/kgPuf', 'mol/molPuf',
       'g/gUi', 'g/gU238f', 'g/tUi', 'mg/gUf', 'g/tHMi', 'mol/molU238i',
       'g/gHMi', 'mol/molUi', 'μg/gUi', 'μg/gUf', 'mg/gU238f', 'g/gU238i',
       'g/gUf', 'mol/molHMi', 'mg/kgUi', 'mg/tUi', 'g/kgUi', 'g/gPuf',
       'mg/gHMi'], dtype=object)

In [115]:
conc.loc[conc.Unit == 'kg/kgPuf']

,ReactorName,ReactorType,Burnup,Enrichment,SampleRef,Measurement,Isotope,Value,Unit,Concentration,ConcentrationUnit,Sigma,Uncertainty,BurnupUnits,EnrichmentUnits
901,Mihama-3,PWR,6.9,3.208,MIH-3|JPNNM3SFA1|E1|86B03,Isotopic Concentration,Pu238,0.00104,kg/kgPuf,NaN,NaN,NaN,NaN,GW*d/tUi,wt%
905,Mihama-3,PWR,6.9,3.208,MIH-3|JPNNM3SFA1|E1|86B03,Isotopic Concentration,Pu239,0.86600,kg/kgPuf,NaN,NaN,NaN,NaN,GW*d/tUi,wt%
908,Mihama-3,PWR,6.9,3.208,MIH-3|JPNNM3SFA1|E1|86B03,Isotopic Concentration,Pu240,0.10500,kg/kgPuf,NaN,NaN,NaN,NaN,GW*d/tUi,wt%
912,Mihama-3,PWR,6.9,3.208,MIH-3|JPNNM3SFA1|E1|86B03,Isotopic Concentration,Pu241,0.02540,kg/kgPuf,NaN,NaN,NaN,NaN,GW*d/tUi,wt%
915,Mihama-3,PWR,6.9,3.208,MIH-3|JPNNM3SFA1|E1|86B03,Isotopic Concentration,Pu242,0.00185,kg/kgPuf,NaN,NaN,NaN,NaN,GW*d/tUi,wt%
966,Mihama-3,PWR,8.3,3.208,MIH-3|JPNNM3SFA1|E1|86B02,Isotopic Concentration,Pu238,0.00126,kg/kgPuf,NaN,NaN,NaN,NaN,GW*d/tUi,wt%
970,Mihama-3,PWR,8.3,3.208,MIH-3|JPNNM3SFA1|E1|86B02,Isotopic Concentration,Pu239,0.84700,kg/kgPuf,NaN,NaN,NaN,NaN,GW*d/tUi,wt%
973,Mihama-3,PWR,8.3,3.208,MIH-3|JPNNM3SFA1|E1|86B02,Isotopic Concentration,Pu240,0.11800,kg/kgPuf,NaN,NaN,NaN,NaN,GW*d/tUi,wt%
977,Mihama-3,PWR,8.3,3.208,MIH-3|JPNNM3SFA1|E1|86B02,Isotopic Concentration,Pu241,0.03040,kg/kgPuf,NaN,NaN,NaN,NaN,GW*d/tUi,wt%
980,Mihama-3,PWR,8.3,3.208,MIH-3|JPNNM3SFA1|E1|86B02,Isotopic Concentration,Pu242,0.00266,kg/kgPuf,NaN,NaN,NaN,NaN,GW*d/tUi,wt%


In [116]:
### Other measurement types ###
act = fulldb.loc[fulldb['Measurement'] == 'Activity'].copy()
#mass = fulldb.loc[fulldb['Measurement'] == 'Mass Ratio'].copy()
#atom = fulldb.loc[fulldb['Measurement'] == 'Atom Ratio'].copy()
act.Unit.unique()

array(['Ci/gFueli', 'Bq/gUi', 'Bq/gUf', 'Ci/gUi', 'Ci/gUf', 'Ci/gHMi',
       'Bq/kgUi'], dtype=object)